In [1]:
from __future__ import print_function
import os
from skimage import io, transform
import numpy as np
from tqdm import tqdm
from model import FireNet

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from torch.utils.tensorboard import SummaryWriter

In [2]:
writer = SummaryWriter('runs/firenet_experiment_1')

In [3]:
print(torch.cuda.is_available())
net = FireNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.cuda()

True


FireNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (drop1): Dropout2d(p=0.5, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=2304, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=2, bias=True)
)

In [4]:
TRAINING_PATH='Dataset'
CATEGORIES = ['Fire', 'NoFire']

In [5]:
class Rescale(object):
    def __init__(self, output_size):
        assert isinstance(output_size, tuple)
        self.output_size = output_size
        
    def __call__(self, image):
        h, w = image.shape[:2]
        new_h, new_w = self.output_size
        new_h, new_w = int(new_h), int(new_w)
        img = transform.resize(image, (new_h, new_w))
        return img
    
class ToTensor(object):
    def __call__(self, image):
        image = image.transpose((2, 0, 1))
        return torch.from_numpy(image)

In [6]:
class TrainingSet(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        classes = []
        images = []
        for category in CATEGORIES:
            path = os.path.join(TRAINING_PATH, category)
            class_num = CATEGORIES.index(category)
            
            for img in tqdm(os.listdir(path)):
                try:
                    image = io.imread(os.path.join(path, img))
                    if (image.shape[2] == 3):
                        if (self.transform):
                            composed = transforms.Compose([Rescale((64, 64)), ToTensor()])
                            image = composed(image)
                            images.append(image)
                            classes.append(class_num)
                except Exception as e:
                    pass
        self.set = {'image': images, 'class': classes}
    def __len__(self):
        return len(self.set['class'])
    def __getitem__(self, idx):
        image = self.set['image'][idx]
        classe = self.set['class'][idx]
        sample = {'image': image, 'class': classe}
        return sample
            
        

In [7]:
training_set= TrainingSet(transform=True)

100%|██████████| 1301/1301 [08:09<00:00,  2.66it/s] 


In [8]:
class TestingSet(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        classes = []
        images = []
        for category in CATEGORIES:
            path = os.path.join('Test_Dataset', category)
            class_num = CATEGORIES.index(category)
            
            for img in tqdm(os.listdir(path)):
                try:
                    image = io.imread(os.path.join(path, img))
                    if (image.shape[2] == 3):
                        if (self.transform):
                            composed = transforms.Compose([Rescale((64, 64)), ToTensor()])
                            image = composed(image)
                            images.append(image)
                            classes.append(class_num)
                except Exception as e:
                    pass
        self.set = {'image': images, 'class': classes}
    def __len__(self):
        return len(self.set['class'])
    def __getitem__(self, idx):
        image = self.set['image'][idx]
        classe = self.set['class'][idx]
        sample = {'image': image, 'class': classe}
        return sample
            
        

In [9]:
test_set = TestingSet(transform=True)

100%|██████████| 100/100 [00:42<00:00,  2.35it/s]


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [11]:
trainloader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=4)
net = net.float()

In [12]:
len(trainloader)

75

In [13]:
for epoch in range(100):
    global_loss = 0.0
    for data in tqdm(trainloader):
        inputs, labels = data['image'].cuda(), data['class'].cuda()

        optimizer.zero_grad()

        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        global_loss += loss.item()
    print('global loss for epoch %d : %.3f' % (epoch + 1, global_loss/ len(trainloader)))
    #writer.add_scalar('Training loss 300', global_loss / len(trainloader), epoch + 1)

  0%|          | 0/75 [00:00<?, ?it/s]

global loss for epoch 1 : 0.646


  0%|          | 0/75 [00:00<?, ?it/s]

global loss for epoch 2 : 0.588


  0%|          | 0/75 [00:00<?, ?it/s]

...

global loss for epoch 98 : 0.186


  0%|          | 0/75 [00:00<?, ?it/s]

global loss for epoch 99 : 0.206


100%|██████████| 75/75 [00:00<00:00, 103.25it/s]

global loss for epoch 100 : 0.198


In [14]:
torch.save(net.state_dict(), './trained_weights100.pth')

In [15]:
net = FireNet()
net.float()
net.cuda()
net.load_state_dict(torch.load('./trained_weights300.pth'))

<All keys matched successfully>

In [16]:
trainloader = DataLoader(training_set, batch_size=4, shuffle=False, num_workers=0)
net = net.float()

In [17]:
correct = 0
total = 0
with torch.no_grad():
    print('evaluate accuracy on training set:')
    for data in tqdm(trainloader):
        images, labels = data['image'].to(device), data['class'].to(device)
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
print ('accuracy: %d' % (100*correct/total))

 17%|█▋        | 102/600 [00:00<00:00, 1017.08it/s]

evaluate accuracy on training set:


100%|██████████| 600/600 [00:00<00:00, 1032.01it/s]

accuracy: 97


In [18]:
testloader = DataLoader(test_set, batch_size=4, shuffle=False, num_workers=0)
net = net.float()

In [19]:
len(testloader)

50

In [20]:
correct = 0
total = 0
with torch.no_grad():
    print('evaluate accuracy on training set:')
    for data in tqdm(testloader):
        images, labels = data['image'].to(device), data['class'].to(device)
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
print ('accuracy: %d' % (100*correct/total))

100%|██████████| 50/50 [00:00<00:00, 925.90it/s]

evaluate accuracy on training set:
accuracy: 68
